# Benchmarks

## Initialize

In [ ]:
%load_ext autoreload
%autoreload 2

import os
import math
import pathlib
import numpy as np
import pandas as pd
from tqdm.auto import tqdm
from IPython.display import clear_output

import warnings
from lifelines.utils import CensoringType
from lifelines.utils import concordance_index

In [ ]:
node = !hostname
if "sc" in node[0]:
    base_path = "/sc-projects/sc-proj-ukb-cvd"
else: 
    base_path = "/data/analysis/ag-reils/ag-reils-shared/cardioRS"
print(base_path)

project_label = "22_medical_records"
project_path = f"{base_path}/results/projects/{project_label}"
figure_path = f"{project_path}/figures"
output_path = f"{project_path}/data"

pathlib.Path(figure_path).mkdir(parents=True, exist_ok=True)
pathlib.Path(output_path).mkdir(parents=True, exist_ok=True)

experiment = 220627
experiment_path = f"{output_path}/{experiment}"
pathlib.Path(experiment_path).mkdir(parents=True, exist_ok=True)

In [ ]:
data_outcomes = pd.read_feather(f"{output_path}/baseline_outcomes_220627.feather").set_index("eid")
endpoints = [e[:-6] for e in data_outcomes.columns if "_event" in e]
del data_outcomes

In [ ]:
endpoint_defs = pd.read_feather(f"{output_path}/phecode_defs_220306.feather").query("endpoint==@endpoints").sort_values("endpoint").set_index("endpoint")

In [ ]:
data_covariates = pd.read_feather("/sc-projects/sc-proj-ukb-cvd/data/2_datasets_pre/211110_anewbeginning/baseline_covariates_211209.feather")[["eid", "sex_f31_0_0"]].set_index("eid")

In [ ]:
data_outcomes = pd.read_feather(f"{output_path}/baseline_outcomes_long_220627.feather").set_index("eid")

In [ ]:
data_all = data_outcomes.merge(data_covariates, left_index=True, right_index=True, how="left").reset_index(drop=False).set_index("endpoint")

In [ ]:
data_all.head()

In [ ]:
data_dict = {e: df.reset_index(drop=True).set_index("eid") for e, df in data_all.groupby('endpoint')}

In [ ]:
endpoint_defs.sex.unique()

In [ ]:
def get_eligable_eids(data_dict, endpoint):

    data_temp = data_dict[endpoint]
    eligibility = endpoint_defs.loc[endpoint]["sex"]
    
    if eligibility == "Both": 
        eids_incl = data_temp.copy().query(f"prev==0").index.to_list()
    else:
        eids_incl = data_temp.copy().query(f"prev==0&sex_f31_0_0==@eligibility").index.to_list()
        
    return {"endpoint": endpoint, 
            "n_eids": len(eids_incl), 
            "eid_list": eids_incl}

In [ ]:
d_list = [get_eligable_eids(data_dict, endpoint) for endpoint in tqdm(endpoints)]
eid_df = pd.DataFrame.from_dict(d_list)

In [ ]:
#eid_df.set_index("endpoint")["eid_list"].to_dict()

In [ ]:
eid_df.to_feather(f"{output_path}/eligable_eids_220627.feather")

In [ ]:
eid_df_long = eid_df[["endpoint", "eid_list"]].explode("eid_list").reset_index(drop=True)
eid_df_long.columns = ["endpoint", "eid"]
eid_df_long["endpoint"] = eid_df_long["endpoint"].astype("category")
eid_df_long["eid"] = eid_df_long["eid"].astype("category")

In [ ]:
eid_df_long.to_feather(f"{output_path}/eligable_eids_long_220627.feather")